In [ ]:
!pip install transformers accelerate peft trl datasets chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s et

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# train_grpo.py
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

In [ ]:
raw_dataset = load_dataset("tapxc3/owast_new", split="train")

In [ ]:
print(raw_dataset[0])

{'id': 'fe00602d-677e-442d-9f7f-7634b105d07d', 'status': 'completed', 'inserted_at': datetime.datetime(2025, 5, 23, 13, 30, 58, 556312), 'updated_at': datetime.datetime(2025, 5, 27, 10, 15, 31, 380335), '_server_id': '20e4deb4-45dc-45ec-a2f2-ef585cd66061', 'prompt': 'Discuss the transformation of Snowball and Napoleon’s ideologies post-rebellion and their impact on the governance model for a paragraph response. from Animal Farm book.', 'result': 'Question\n\nHow do Snowball and Napoleon\'s ideologies evolve post-rebellion, and what impact does this have on the governance model of Animal Farm?\n\nSupporting context\n\nBy the late summer, Snowball and Napoleon were actively spreading the ideology of the rebellion by sending out pigeons to neighboring farms to share the story and teach the song "Beasts of England" to other animals. This shows their initial commitment to spreading revolutionary ideals and uniting animals against human oppressors. However, as time progresses, their ideologi

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch
from trl import SFTTrainer, GRPOConfig
from trl import SFTConfig

from chromadb import HttpClient
from chromadb.config import Settings

import json
from glob import glob
import os
import spacy
from spacy.matcher import PhraseMatcher
import re

chroma_client = HttpClient(host="34.126.200.250", port=8000)
collection = chroma_client.get_collection(name="orwell_books")

In [ ]:
# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Known book titles
BOOK_TITLES = [
    "Nineteen Eighty-Four", "Animal Farm", "Shooting an Elephant", "Homage to Catalonia",
    "Coming Up for Air", "Down and Out in Paris and London", "Keep the Aspidistra Flying",
    "A Clergyman's Daughter", "Fifty Orwell Essays", "Politics and the English Language",
    "The Road to Wigan Pier", "Burmese Days"
]

# Maps for conversion
spelled_to_digit = {
    "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6,
    "seven": 7, "eight": 8, "nine": 9, "ten": 10, "eleven": 11,
    "twelve": 12, "thirteen": 13, "fourteen": 14, "fifteen": 15,
    "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19, "twenty": 20
}

digit_to_spelled = {
    1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six",
    7: "seven", 8: "eight", 9: "nine", 10: "ten", 11: "eleven",
    12: "twelve", 13: "thirteen", 14: "fourteen", 15: "fifteen",
    16: "sixteen", 17: "seventeen", 18: "eighteen", 19: "nineteen", 20: "twenty"
}

digit_to_roman = {
    1: "i", 2: "ii", 3: "iii", 4: "iv", 5: "v",
    6: "vi", 7: "vii", 8: "viii", 9: "ix", 10: "x",
    11: "xi", 12: "xii", 13: "xiii", 14: "xiv", 15: "xv",
    16: "xvi", 17: "xvii", 18: "xviii", 19: "xix", 20: "xx"
}

ordinal_to_digit = {
    "first": 1, "second": 2, "third": 3, "fourth": 4, "fifth": 5, "sixth": 6,
    "seventh": 7, "eighth": 8, "ninth": 9, "tenth": 10, "eleventh": 11,
    "twelfth": 12, "thirteenth": 13, "fourteenth": 14, "fifteenth": 15,
    "sixteenth": 16, "seventeenth": 17, "eighteenth": 18, "nineteenth": 19, "twentieth": 20
}

roman_to_int = {
    'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000
}

# Prepare matcher
phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(title) for title in BOOK_TITLES]
phrase_matcher.add("BOOK_TITLE", patterns)

In [ ]:
def extract_metadata(prompt):
    metadata = {"$and": [{"title": None}]}
    doc = nlp(prompt)
    part_no = get_part_number(prompt)
    name = None
    # Match book title using PhraseMatcher
    matches = phrase_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        metadata["$and"][0]["title"] = span.text.lower()
        name = span.text.lower()
        break  # Take the first match
    if part_no is not None:
        digit_word = digit_to_spelled[part_no]
        roman_word = digit_to_roman[part_no]
        chapter_filter = {
            "$or": [
                {"chapter": {"$in": [f"chapter {part_no}", f"part {part_no}", f"chapter {digit_word}", f"part {digit_word}", f"chapter {roman_word}"]}},
                {"subchapter": {"$in": [f"chapter {part_no}", f"part {part_no}", f"chapter {digit_word}", f"part {digit_word}", f"chapter {roman_word}"]}}
            ]
        }
        metadata["$and"].append(chapter_filter)
    else:
        metadata = {"title": name}
    return metadata, name

def retrieve_context(prompt, top_k=1):
    filters, nameBook = extract_metadata(prompt)
    results = collection.query(
        query_texts=["Give me the book named " + nameBook],
        where=filters,
        n_results=top_k
    )

    return [
        {"content": doc, "meta": meta}
        for doc, meta in zip(results['documents'][0], results['metadatas'][0])
    ]

def detect_question_type(user_prompt: str) -> str:
    prompt_lower = user_prompt.lower()

    if any(keyword in prompt_lower for keyword in ["mcq", "multiple choice"]):
        return "MCQs"
    elif any(keyword in prompt_lower for keyword in ["essay", "long answer"]):
        return "Essay"
    elif any(keyword in prompt_lower for keyword in ["short answer"]):
        return "Short Answer"
    elif any(keyword in prompt_lower for keyword in ["paragraph", "paragraph answer"]):
        return "Paragraph Answer"
    elif any(keyword in prompt_lower for keyword in ["passage", "passage-based"]):
        return "Passage-Based"
    elif any(keyword in prompt_lower for keyword in ["quiz"]):
        return "Quiz"
    else:
        return "Short Answer"  # Default fallback

def roman_to_number(roman):
    total = 0
    prev_value = 0
    for char in reversed(roman.upper()):
        value = roman_to_int.get(char, 0)
        if value < prev_value:
            total -= value
        else:
            total += value
        prev_value = value
    return total if total > 0 else None

def get_part_number(prompt):
    doc = nlp(prompt)
    part_keywords = {"chapter", "part", "section"}
    number_pattern = re.compile(r"^\d+$", re.IGNORECASE)
    roman_pattern = re.compile(r"^[IVXLCDM]+$", re.IGNORECASE)

    for i, token in enumerate(doc):
        token_lower = token.text.lower()

        # Case: number or ordinal before keyword
        if token_lower in spelled_to_digit or token_lower in ordinal_to_digit or roman_pattern.match(token_lower) or number_pattern.match(token_lower):
            for j in range(1, 3):
                if i + j < len(doc) and doc[i + j].text.lower() in part_keywords:
                    if token_lower in spelled_to_digit:
                        return spelled_to_digit[token_lower]
                    elif token_lower in ordinal_to_digit:
                        return ordinal_to_digit[token_lower]
                    elif number_pattern.match(token_lower):
                        return int(token_lower)
                    elif roman_pattern.match(token_lower):
                        return roman_to_number(token_lower)

        # Case: keyword before number or ordinal
        elif token_lower in part_keywords:
            for j in range(1, 4):
                if i + j < len(doc):
                    next_token = doc[i + j].text.lower()
                    if next_token in spelled_to_digit:
                        return spelled_to_digit[next_token]
                    elif next_token in ordinal_to_digit:
                        return ordinal_to_digit[next_token]
                    elif number_pattern.match(next_token):
                        return int(next_token)
                    elif roman_pattern.match(next_token):
                        return roman_to_number(next_token)

    return None


def get_prompt(contexts, user_prompt: str = ""):
    question_type = detect_question_type(user_prompt)

    citation_instructions = """
      - Every question and explanation must include at least one **inline citation** in the format: [1]
      - Each inline citation must correspond to a **source reference at the end** of the response, using this exact format:
        [1] Book Title, Chapter X
      - Use numeric citations ([1], [2], etc.) in square brackets.
      - Place citations **immediately after the sentence they support**.
      - Do NOT use author-date style or URLs.
    """

    if question_type == "MCQs":
        output_format = """
          Question:

          Answer options:
            A. Option A
            B. Option B
            C. Option C
            D. Option D

          Supporting content:

          Sources:
          [1] Book Title, Chapter X
        """
    else:
        output_format = """
          Question:

          Supporting content:

          Sources:
          [1] Book Title, Chapter X
        """

    # Get the appropriate output format or fall back
    # output_format = format_templates.get(question_type, format_templates)

    prompt = f"""
      You are a literature teaching assistant. When given a prompt that references a literary element,
      generate a question and retrieve supporting content from the context (quote, paragraph, passage, or poem).

      Instructions:
      - Output only **1 {question_type} question** per request.
      - If it's a multiple-choice question, list each choice directly below the question (one per line).
      - After the question include a **supporting content** and must be less than 200 words. This may be:
          - A sentence
          - A short paragraph
          - A direct quote
          - A passage
          - A poem or poetic excerpt
      - Choose the most relevant textual evidence that directly supports the question
      - Follow this citation format exactly:{citation_instructions}

      Context for generate question:
      {contexts}

      Now generate the output exactly in the format format:
      {output_format}

      Now Complete:
    """
    return prompt

In [ ]:
def make_format(example):
    return {
        "prompt": [
            {"role": "system", "content": get_prompt(retrieve_context(example["prompt"]), example["prompt"])},
            {"role": "user", "content": example["prompt"]},
        ],
    }

In [ ]:
train_dataset = raw_dataset.map(make_format)

Parameter 'function'=<function make_format at 0x7cc18afef600> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns([
    'id', 'status', 'inserted_at', 'updated_at', '_server_id',
    'intent_relevance.responses.users', 'intent_relevance.responses.status',
    'citation_support.responses.users', 'citation_support.responses.status',
    'hallucination_check.responses.users', 'hallucination_check.responses.status',
    'clarity_rating.responses.users', 'clarity_rating.responses.status',
    'relevance_rating.responses.users', 'relevance_rating.responses.status',
    'overall_quality.responses.users', 'overall_quality.responses.status'])

ValueError: Column name ['id', 'overall_quality.responses.users', 'hallucination_check.responses.status', 'hallucination_check.responses.users', 'citation_support.responses.users', 'clarity_rating.responses.status', 'relevance_rating.responses.status', 'inserted_at', 'updated_at', 'citation_support.responses.status', 'intent_relevance.responses.status', 'relevance_rating.responses.users', 'overall_quality.responses.status', 'status', 'intent_relevance.responses.users', '_server_id', 'clarity_rating.responses.users'] not in the dataset. Current columns in the dataset: ['prompt', 'result', 'intent_relevance.responses', 'citation_support.responses', 'hallucination_check.responses', 'clarity_rating.responses', 'relevance_rating.responses', 'overall_quality.responses']

In [ ]:
print(train_dataset[0])

{'prompt': [{'content': '\n      You are a literature teaching assistant. When given a prompt that references a literary element,\n      generate a question and retrieve supporting content from the context (quote, paragraph, passage, or poem).\n\n      Instructions:\n      - Output only **1 Paragraph Answer question** per request.\n      - If it\'s a multiple-choice question, list each choice directly below the question (one per line).\n      - After the question include a **supporting content** and must be less than 200 words. This may be:\n          - A sentence\n          - A short paragraph\n          - A direct quote\n          - A passage\n          - A poem or poetic excerpt\n      - Choose the most relevant textual evidence that directly supports the question\n      - Follow this citation format exactly:\n      - Every question and explanation must include at least one **inline citation** in the format: [1]\n      - Each inline citation must correspond to a **source reference a

In [ ]:
import torch
from transformers import AutoModelForCausalLM

In [ ]:
model_id = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 59,867,136 || all params: 3,145,805,824 || trainable%: 1.9031


In [ ]:
def binary_reward(value):
    return 1.0 if value == "yes" else -1.0

def normalize_rating(value, max_value=5):
    return value / max_value

def compute_reward(sample):
    """
    Computes a scalar reward from a single dataset sample using human feedback fields.
    """

    # Get values with safe defaults
    intent = binary_reward(sample.get("intent_relevance.responses", ["no"])[0])
    citation = binary_reward(sample.get("citation_support.responses", ["no"])[0])
    hallucination = binary_reward(sample.get("hallucination_check.responses", ["no"])[0])

    clarity = normalize_rating(sample.get("clarity_rating.responses", [0])[0])
    relevance = normalize_rating(sample.get("relevance_rating.responses", [0])[0])
    overall = normalize_rating(sample.get("overall_quality.responses", [0])[0])

    # Weighted reward calculation
    reward = (
        0.2 * intent +
        0.2 * citation +
        0.2 * hallucination +
        0.1 * clarity +
        0.15 * relevance +
        0.15 * overall
    )

    return reward


In [ ]:
def reward_calculate(completions, **kwargs):
    rewards = kwargs["reward"]
    return rewards

In [ ]:
from datasets import Dataset

# If train_dataset is a Hugging Face Dataset
def add_reward(example):
    example["reward"] = compute_reward(example)
    return example

train_dataset = train_dataset.map(add_reward)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    output_dir=model_id + "_test3",
    learning_rate=2e-5,
    remove_unused_columns=False, # to access the solution column in accuracy_reward
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    bf16=True,

    # Parameters that control de data preprocessing
    max_completion_length=96, # default: 256
    num_generations=2, # default: 8
    max_prompt_length=256, # default: 512

    # Parameters related to reporting and saving
    report_to=["tensorboard"],
    logging_steps=10,
    push_to_hub=True,
    save_strategy="steps",
    save_steps=50,
)

In [ ]:
from trl import GRPOTrainer
from transformers import default_data_collator

trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_calculate,
    args=training_args,
    train_dataset=train_dataset
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
10,0.000000


TrainOutput(global_step=15, training_loss=0.0, metrics={'train_runtime': 342.5641, 'train_samples_per_second': 3.153, 'train_steps_per_second': 0.053, 'total_flos': 0.0, 'train_loss': 0.0})

In [ ]:
dataset_id = "tapxc3/owast_new"
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name=dataset_id)

NameError: name 'trainer' is not defined

Question Generation Start Here

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "tapxc3/Qwen2.5-3B-Instruct_test2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)
tokenizer.pad_token = tokenizer.eos_token  # ensure padding works

In [ ]:
from chromadb import HttpClient
from chromadb.config import Settings

import json
from glob import glob
import os
import spacy
from spacy.matcher import PhraseMatcher
import re

chroma_client = HttpClient(host="34.126.200.250", port=8000)
collection = chroma_client.get_collection(name="orwell_books")

In [ ]:
# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Known book titles
BOOK_TITLES = [
    "Nineteen Eighty-Four", "Animal Farm", "Shooting an Elephant", "Homage to Catalonia",
    "Coming Up for Air", "Down and Out in Paris and London", "Keep the Aspidistra Flying",
    "A Clergyman's Daughter", "Fifty Orwell Essays", "Politics and the English Language",
    "The Road to Wigan Pier", "Burmese Days"
]

# Maps for conversion
spelled_to_digit = {
    "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6,
    "seven": 7, "eight": 8, "nine": 9, "ten": 10, "eleven": 11,
    "twelve": 12, "thirteen": 13, "fourteen": 14, "fifteen": 15,
    "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19, "twenty": 20
}

digit_to_spelled = {
    1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six",
    7: "seven", 8: "eight", 9: "nine", 10: "ten", 11: "eleven",
    12: "twelve", 13: "thirteen", 14: "fourteen", 15: "fifteen",
    16: "sixteen", 17: "seventeen", 18: "eighteen", 19: "nineteen", 20: "twenty"
}

digit_to_roman = {
    1: "i", 2: "ii", 3: "iii", 4: "iv", 5: "v",
    6: "vi", 7: "vii", 8: "viii", 9: "ix", 10: "x",
    11: "xi", 12: "xii", 13: "xiii", 14: "xiv", 15: "xv",
    16: "xvi", 17: "xvii", 18: "xviii", 19: "xix", 20: "xx"
}

ordinal_to_digit = {
    "first": 1, "second": 2, "third": 3, "fourth": 4, "fifth": 5, "sixth": 6,
    "seventh": 7, "eighth": 8, "ninth": 9, "tenth": 10, "eleventh": 11,
    "twelfth": 12, "thirteenth": 13, "fourteenth": 14, "fifteenth": 15,
    "sixteenth": 16, "seventeenth": 17, "eighteenth": 18, "nineteenth": 19, "twentieth": 20
}

roman_to_int = {
    'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000
}

# Prepare matcher
phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(title) for title in BOOK_TITLES]
phrase_matcher.add("BOOK_TITLE", patterns)

In [ ]:
def extract_metadata(prompt):
    metadata = {"$and": [{"title": None}]}
    doc = nlp(prompt)
    part_no = get_part_number(prompt)
    name = None
    # Match book title using PhraseMatcher
    matches = phrase_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        metadata["$and"][0]["title"] = span.text.lower()
        name = span.text.lower()
        break  # Take the first match
    if part_no is not None:
        digit_word = digit_to_spelled[part_no]
        roman_word = digit_to_roman[part_no]
        chapter_filter = {
            "$or": [
                {"chapter": {"$in": [f"chapter {part_no}", f"part {part_no}", f"chapter {digit_word}", f"part {digit_word}", f"chapter {roman_word}"]}},
                {"subchapter": {"$in": [f"chapter {part_no}", f"part {part_no}", f"chapter {digit_word}", f"part {digit_word}", f"chapter {roman_word}"]}}
            ]
        }
        metadata["$and"].append(chapter_filter)
    else:
        metadata = {"title": name}
    return metadata, name

def retrieve_context(prompt, top_k=1):
    filters, nameBook = extract_metadata(prompt)
    results = collection.query(
        query_texts=["Give me the book named " + nameBook],
        where=filters,
        n_results=top_k
    )

    return [
        {"content": doc, "meta": meta}
        for doc, meta in zip(results['documents'][0], results['metadatas'][0])
    ]

def detect_question_type(user_prompt: str) -> str:
    prompt_lower = user_prompt.lower()

    if any(keyword in prompt_lower for keyword in ["mcq", "multiple choice"]):
        return "MCQs"
    elif any(keyword in prompt_lower for keyword in ["essay", "long answer"]):
        return "Essay"
    elif any(keyword in prompt_lower for keyword in ["short answer"]):
        return "Short Answer"
    elif any(keyword in prompt_lower for keyword in ["paragraph", "paragraph answer"]):
        return "Paragraph Answer"
    elif any(keyword in prompt_lower for keyword in ["passage", "passage-based"]):
        return "Passage-Based"
    elif any(keyword in prompt_lower for keyword in ["quiz"]):
        return "Quiz"
    else:
        return "Short Answer"  # Default fallback

def roman_to_number(roman):
    total = 0
    prev_value = 0
    for char in reversed(roman.upper()):
        value = roman_to_int.get(char, 0)
        if value < prev_value:
            total -= value
        else:
            total += value
        prev_value = value
    return total if total > 0 else None

def get_part_number(prompt):
    doc = nlp(prompt)
    part_keywords = {"chapter", "part", "section"}
    number_pattern = re.compile(r"^\d+$", re.IGNORECASE)
    roman_pattern = re.compile(r"^[IVXLCDM]+$", re.IGNORECASE)

    for i, token in enumerate(doc):
        token_lower = token.text.lower()

        # Case: number or ordinal before keyword
        if token_lower in spelled_to_digit or token_lower in ordinal_to_digit or roman_pattern.match(token_lower) or number_pattern.match(token_lower):
            for j in range(1, 3):
                if i + j < len(doc) and doc[i + j].text.lower() in part_keywords:
                    if token_lower in spelled_to_digit:
                        return spelled_to_digit[token_lower]
                    elif token_lower in ordinal_to_digit:
                        return ordinal_to_digit[token_lower]
                    elif number_pattern.match(token_lower):
                        return int(token_lower)
                    elif roman_pattern.match(token_lower):
                        return roman_to_number(token_lower)

        # Case: keyword before number or ordinal
        elif token_lower in part_keywords:
            for j in range(1, 4):
                if i + j < len(doc):
                    next_token = doc[i + j].text.lower()
                    if next_token in spelled_to_digit:
                        return spelled_to_digit[next_token]
                    elif next_token in ordinal_to_digit:
                        return ordinal_to_digit[next_token]
                    elif number_pattern.match(next_token):
                        return int(next_token)
                    elif roman_pattern.match(next_token):
                        return roman_to_number(next_token)

    return None


def get_prompt(contexts, user_prompt: str = ""):
    question_type = detect_question_type(user_prompt)

    citation_instructions = """
      - Every question and explanation must include at least one **inline citation** in the format: [1]
      - Each inline citation must correspond to a **source reference at the end** of the response, using this exact format:
        [1] Book Title, Chapter X
      - Use numeric citations ([1], [2], etc.) in square brackets.
      - Place citations **immediately after the sentence they support**.
      - Do NOT use author-date style or URLs.
    """

    if question_type == "MCQs":
        output_format = """
          Question:

          Answer options:
            A. Option A
            B. Option B
            C. Option C
            D. Option D

          Supporting content:

          Sources:
          [1] Book Title, Chapter X
        """
    else:
        output_format = """
          Question:

          Supporting content:

          Sources:
          [1] Book Title, Chapter X
        """

    # Get the appropriate output format or fall back
    # output_format = format_templates.get(question_type, format_templates)

    prompt = f"""
      You are a literature teaching assistant. When given a prompt that references a literary element,
      generate a question and retrieve supporting content from the context (quote, paragraph, passage, or poem).

      Instructions:
      - Output only **1 {question_type} question** per request.
      - If it's a multiple-choice question, list each choice directly below the question (one per line).
      - After the question include a **supporting content** and must be less than 200 words. This may be:
          - A sentence
          - A short paragraph
          - A direct quote
          - A passage
          - A poem or poetic excerpt
      - Choose the most relevant textual evidence that directly supports the question
      - Follow this citation format exactly:{citation_instructions}

      Context for generate question:
      {contexts}

      Now generate the output exactly in the format format:
      {output_format}

      Now Complete:
    """
    return prompt

In [ ]:
def generate_response(prompt):
    final_prompt = f"""You are teacher assistance helping me create a question using these context:
{str(retrieve_context(user_prompt))}
I want you to create only one question from this prompt: {user_prompt}.
Your have to include only exact 3 following parts
1. A question (include choices only if this is MCQ or multiple choices. Anything else is open-end question)

2. A supporting content

3. A source reference (only book title and chapter in "[1] Book title, chapter" formant)

You must generate follow this format:
Question ->
Support Content ->
Source -> [1] Book title, chapter _

You need to include all and only the 3 above parts in the response.
Must create only 1 question.
"""
        # Tokenize input
    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)

    # Generate output
    output = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    output = decoded.removeprefix(final_prompt)
    return output

In [ ]:
# Change Prompt Here
user_prompt = "Generate a multiple choices question about Snowball's mental state in Animal Farm."
print(generate_response(user_prompt))

Question -> Which of the following best describes Snowball's mental state during the Battle of the Cowshed?
A) Overconfident and eager to lead the charges
B) Calm and strategic, giving orders to the animals
C) Fearful and reluctant to fight, preferring to hide
D) Angry and vengeful, seeking to harm the humans
Support Content -> Throughout the battle, Snowball demonstrated a calm and strategic approach to leading the animals. He gave precise orders and coordinated the defense effectively. Despite the ferocity of the fight, Snowball remained composed, directing the animals to their positions and ensuring they were ready to counterattack. However, after the battle, he expressed a harsher tone, criticizing the humans and suggesting that war is inherently brutal.
Source -> [1] Animal Farm, chapter IV _
